In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import logging

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import requests
import time
import json
import re
import collections
import torch
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as torch_func
from train_transformer import HangmanTransformer, Vocab
logger = logging.getLogger(__name__)



In [ ]:
def guess_model_transformer(model, vocab, masked_input, guessed_letters):
    model.eval()
    encoded = vocab.encode(masked_input)
    input_tensor = torch.tensor(encoded, dtype=torch.long).unsqueeze(0)
    attention_mask = (input_tensor != 0)

    #to track the weighting consistently, and not backtrack on the weight, and not using gradients
    with torch.no_grad():
        output = model(input_tensor, mask=torch.bitwise_not(attention_mask))[0]
        probs = torch_func.softmax(output, dim=-1)

    masked_positions = []
    for i, val in enumerate(encoded):
        if val == 0:
            masked_positions.append(i)

    #creates evenly numbers in the range of 1.5-2, being a parameter to focus on the suffix methodology
    # then we assign 0s to the matrix, and after, we are creating the probability for each position in the index. 
    position_weights = np.linspace(1.5, 2, len(masked_positions))
    letter_scores = torch.zeros(probs.shape[-1])

    for weight, pos in zip(position_weights, masked_positions):
        letter_scores += weight * probs[pos]

    #Build the list of (index, score) pairs and sorting the letter_scores
    letter_scores = letter_scores.numpy()

    indexed_scores = []
    for i in range(len(letter_scores)):
        #logger.info(f"i in letters : {i}")
        indexed_scores.append((i, letter_scores[i]))
    sorted_indexed_scores = sorted(indexed_scores, key=lambda x: -x[1])

    sorted_indices = [idx for idx, _ in sorted_indexed_scores]
    sorted_scores = [score for _, score in sorted_indexed_scores]
    cumulative_probability = []
    total = 0.0
    for score in sorted_scores:
        total += score
        cumulative_probability.append(total)

    probability_cutoff = 0.9 
    cutoff = 0
    for i, cp in enumerate(cumulative_probability):
        if cp >= probability_cutoff:
            cutoff = i
            break

    top_indices_select = sorted_indices[:cutoff + 1]

    #removing guesseding letters
    available_indices = [i for i in top_indices_select if vocab.index_character.get(i, '') not in guessed_letters]

    if not available_indices:
        for i in sorted_indices:
            ch = vocab.index_character.get(i, '')
            if ch not in guessed_letters:
                return ch
    
    sampled_index = random.choice(available_indices)
    return vocab.index_character.get(sampled_index, '_')

In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "xxx.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []

        self.model, self.vocab = self.load_transformer()

    def load_transformer(self):
        with open("xxx.txt") as f:
            words = [line.strip().lower() for line in f]
        vocab = Vocab(words)
        vocab_size = max(vocab.character_index.values()) + 1
        model = HangmanTransformer(vocab_size)
        model.load_state_dict(torch.load("hangman_transformer.pt", map_location=torch.device('cpu')))
        model.eval()
        return model, vocab
    def guess(self, word): # word input example: "_ p p _ e "
        word = word.lower().replace(" ", "")
        
        #suffixes methodology
        if word.endswith("___") or word.endswith("__") or word.endswith("_"):
            for suffixes_letter in ['s', 'e', 'd', 'y', 'r', 'n', 'g', 't', 'x']:
                if suffixes_letter not in self.guessed_letters:
                    return suffixes_letter
                
        guess_letter = guess_model_transformer(
            model=self.model,
            vocab=self.vocab,
            masked_input=word,
            guessed_letters=self.guessed_letters
        )
        return guess_letter
    
    #DEFINE YOUR FUNCTION TO PLAY THE GAME..